In [0]:
!pip freeze | grep scikit-learn

In [0]:
!python -V

In [0]:
import pickle
import pandas as pd

In [0]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [0]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [0]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-04.parquet')

In [0]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [0]:
y_pred  # HW 1

In [0]:
y_pred.std()  # HW 2

In [0]:
y_pred.mean()

In [0]:
year = 2023
month = 4

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df['prediction'] = y_pred


df_result = df[['ride_id', 'prediction']]

display(df_result)


In [0]:
output_file = f'./output-{year:04d}-{month:02d}.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [0]:
%sh
ls -ltrh ./output-2023-03.parquet  # HW 3